In [2]:
import os
os.environ["SPARK_VERSION"] = '3.3'

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1703332378559_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import pydeequ

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1703332378559_0001,pyspark,idle,Link,Link,None,✔


In [1]:
%%configure -f
{
    "conf": {
      "spark.jars.packages": "com.amazon.deequ:deequ:2.0.3-spark-3.3",
      "spark.jars.excludes": "net.sourceforge.f2j:arpack_combined_all"
    }
}

# Reading TLC Yellow Taxi Data from S3

In [5]:
bucket_name = "my_s3_data_bucket"
yellow_df = spark.read.parquet(f"s3://{bucket_name}/raw_data/nyc_tlc/yellow/oct2023/")
yellow_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)

## Profiling Data

In [6]:
from pydeequ.profiles import ColumnProfilerRunner

# Profiling all the columns: ColumnProfilerRunner.onData returns a ColumnProfilerRunBuilder
result = ColumnProfilerRunner(spark) \
            .onData(yellow_df) \
            .run()

# printing all the columns and their corresponding profiled data.
for col_name, profile in result.profiles.items():
    print(col_name)
    print(profile)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DOLocationID
NumericProfiles for column: DOLocationID: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 262,
    "dataType": "Integral",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": null,
    "kll": "None",
    "mean": 164.5522420814897,
    "maximum": 265.0,
    "minimum": 1.0,
    "sum": 579599894.0,
    "stdDev": 69.76912935351677,
    "approxPercentiles": []
}
improvement_surcharge
NumericProfiles for column: improvement_surcharge: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 5,
    "dataType": "Fractional",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": [
        [
            "0.0",
            1539,
            0.00043693227549729795
        ],
        [
            "-1.0",
            36962,
            0.010493756183840887
        ],
        [
            "0.3",
            916,
            0.0002600584563713612
        ],
        [
            "1.0",
            3482867,
            0.98880896

### Profiling specific columns

In [7]:
# Restricting to columns of interest only
column_profiler = ColumnProfilerRunner(spark) \
                            .onData(yellow_df) \
                            .restrictToColumns(['VendorID', 'trip_distance', 'total_amount'])\
                            .run()

for col_name, profile in column_profiler.profiles.items():
    print(col_name)
    print(profile)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VendorID
NumericProfiles for column: VendorID: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 3,
    "dataType": "Integral",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": [
        [
            "2",
            2617320,
            0.7430744530894008
        ],
        [
            "1",
            904463,
            0.25678302579149614
        ],
        [
            "6",
            502,
            0.00014252111910308224
        ]
    ],
    "kll": "None",
    "mean": 1.7437870586849162,
    "maximum": 6.0,
    "minimum": 1.0,
    "sum": 6142115.0,
    "stdDev": 0.4397934656174618,
    "approxPercentiles": []
}
trip_distance
NumericProfiles for column: trip_distance: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 4888,
    "dataType": "Fractional",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": null,
    "kll": "None",
    "mean": 3.926694986351057,
    "maximum": 205544.17,
    "minimum": 0.0,
  

## Analyzers for Data Analysis

In [8]:
from pydeequ.analyzers import AnalysisRunner, AnalyzerContext, Size, Completeness, Distinctness, Uniqueness, Compliance, Mean, Sum, Maximum

# Adding Analyzers for metrics computation
analysisResult = AnalysisRunner(spark) \
                    .onData(yellow_df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("VendorID")) \
                    .addAnalyzer(Distinctness("VendorID")) \
                    .addAnalyzer(Uniqueness(["VendorID", "payment_type"])) \
                    .addAnalyzer(Compliance("payment_type", "payment_type in (1,2,3,4,5,6)")) \
                    .addAnalyzer(Mean("trip_distance")) \
                    .addAnalyzer(Sum("total_amount")) \
                    .addAnalyzer(Maximum("extra")) \
                    .run()
                    
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+------------+--------------------+
|     entity|            instance|        name|               value|
+-----------+--------------------+------------+--------------------+
|     Column|        total_amount|         Sum|1.0274988166009633E8|
|     Column|            VendorID|Distinctness|8.517198352773839E-7|
|Mutlicolumn|VendorID,payment_...|  Uniqueness|                 0.0|
|     Column|       trip_distance|        Mean|   3.926694986351057|
|    Dataset|                   *|        Size|           3522285.0|
|     Column|        payment_type|  Compliance|  0.9560146325467701|
|     Column|            VendorID|Completeness|                 1.0|
|     Column|               extra|     Maximum|               17.25|
+-----------+--------------------+------------+--------------------+

/mnt/yarn/usercache/livy/appcache/application_1703332378559_0001/container_1703332378559_0001_01_000001/pyspark.zip/pyspark/sql/dataframe.py:127: UserWarning: DataFrame co

In [9]:
%%display
analysisResult_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

## Constraints Suggestion

In [12]:
from pydeequ.suggestions import ConstraintSuggestionRunner, DEFAULT
import json

suggestionResult = ConstraintSuggestionRunner(spark) \
             .onData(yellow_df) \
             .addConstraintRule(DEFAULT()) \
             .run()

# Constraint Suggestions in JSON format
print(json.dumps(suggestionResult, indent=2))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{
  "constraint_suggestions": [
    {
      "constraint_name": "CompletenessConstraint(Completeness(DOLocationID,None))",
      "column_name": "DOLocationID",
      "current_value": "Completeness: 1.0",
      "description": "'DOLocationID' is not null",
      "suggesting_rule": "CompleteIfCompleteRule()",
      "rule_description": "If a column is complete in the sample, we suggest a NOT NULL constraint",
      "code_for_constraint": ".isComplete(\"DOLocationID\")"
    },
    {
      "constraint_name": "ComplianceConstraint(Compliance('DOLocationID' has no negative values,DOLocationID >= 0,None))",
      "column_name": "DOLocationID",
      "current_value": "Minimum: 1.0",
      "description": "'DOLocationID' has no negative values",
      "suggesting_rule": "NonNegativeNumbersRule()",
      "rule_description": "If we see only non-negative numbers in a column, we suggest a corresponding constraint",
      "code_for_constraint": ".isNonNegative(\"DOLocationID\")"
    },
    {
      "cons

In [14]:
from pydeequ.suggestions import ConstraintSuggestionRunner, NonNegativeNumbersRule, RetainTypeRule, RetainCompletenessRule
# Running specific suggestion rules, helps in knowing if a particular rule is suggested or not and how it can actually be implemented. 
# Check Airport_fee > RetainCompletenessRule()
specific_suggestion_result = ConstraintSuggestionRunner(spark) \
             .onData(yellow_df) \
             .addConstraintRule(NonNegativeNumbersRule()) \
             .addConstraintRule(RetainTypeRule()) \
             .addConstraintRule(RetainCompletenessRule()) \
             .run()

# Constraint Suggestions in JSON format
print(json.dumps(specific_suggestion_result, indent=2))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{
  "constraint_suggestions": [
    {
      "constraint_name": "ComplianceConstraint(Compliance('DOLocationID' has no negative values,DOLocationID >= 0,None))",
      "column_name": "DOLocationID",
      "current_value": "Minimum: 1.0",
      "description": "'DOLocationID' has no negative values",
      "suggesting_rule": "NonNegativeNumbersRule()",
      "rule_description": "If we see only non-negative numbers in a column, we suggest a corresponding constraint",
      "code_for_constraint": ".isNonNegative(\"DOLocationID\")"
    },
    {
      "constraint_name": "ComplianceConstraint(Compliance('PULocationID' has no negative values,PULocationID >= 0,None))",
      "column_name": "PULocationID",
      "current_value": "Minimum: 1.0",
      "description": "'PULocationID' has no negative values",
      "suggesting_rule": "NonNegativeNumbersRule()",
      "rule_description": "If we see only non-negative numbers in a column, we suggest a corresponding constraint",
      "code_for_constrain

## Constraint Verification

In [15]:
from pyspark.sql.functions import col, monotonically_increasing_id

# Adding a column gen_id to check the rules around uniqueness and column combination uniqueness in Pydeequ
sample_df = yellow_df.withColumn("gen_id", monotonically_increasing_id())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
from pydeequ.checks import *
from pydeequ.verification import *

# Check represents a list of constraints that can be applied to a provided Spark Dataframe
check = Check(spark, CheckLevel.Warning, "NYC Yellow Taxi Trips Oct 2023")

checkResult = VerificationSuite(spark) \
    .onData(sample_df) \
    .addCheck(
        check.isComplete("VendorID")  \
        .isUnique("gen_id")  \
        .hasUniqueness(["VendorID", "gen_id"], assertion=lambda x: x==1) \
        .hasCompleteness("Airport_fee", assertion=lambda x: x >= 0.95) \
        .isNonNegative("fare_amount")) \
    .run()

print(f"Verification Run Status: {checkResult.status}")

# Checking the results of the verification
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Verification Run Status: Warning

In [17]:
%%display
checkResult_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

## Metrics Repositories

In [ ]:
# Initializing Metrics Repository: FileSystemMetricsRepository. Other Repositories are InMemoryMetricsRepository
from pydeequ.repository import FileSystemMetricsRepository, ResultKey

# path to metrics file can be an S3 path or HDFS path.
metrics_file = "s3://{bucket_name}/nyc_tlc/pydeequ_metrics/ny_yellow_metrics.json"

nyc_yellow_repository = FileSystemMetricsRepository(spark, path = metrics_file)
print(metrics_file)

In [35]:
# This tag will uniquely identify the Analysis result
key_tags = {"tag": "nyc_yellow_oct_2023"}
resultKey = ResultKey(spark, ResultKey.current_milli_time(), key_tags)

verify_key_tag = {"tag": "verify_nyc_yellow_oct_2023"}
verify_resultKey = ResultKey(spark, ResultKey.current_milli_time(), verify_key_tag)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
# using .useRepository and saveOrAppendResult for saving results in metrics.json file
analysisResult = AnalysisRunner(spark) \
                    .onData(sample_df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("VendorID")) \
                    .addAnalyzer(Distinctness("VendorID")) \
                    .addAnalyzer(Uniqueness(["VendorID", "gen_id"])) \
                    .addAnalyzer(Compliance("payment_type", "payment_type in (1,2,3,4,5,6)")) \
                    .addAnalyzer(Mean("trip_distance")) \
                    .addAnalyzer(Sum("total_amount")) \
                    .addAnalyzer(Maximum("extra")) \
                    .useRepository(nyc_yellow_repository) \
                    .saveOrAppendResult(resultKey) \
                    .run()
                    
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------------+------------+--------------------+
|     entity|       instance|        name|               value|
+-----------+---------------+------------+--------------------+
|     Column|   total_amount|         Sum|1.0274988166003421E8|
|     Column|       VendorID|Distinctness|8.517198352773839E-7|
|     Column|  trip_distance|        Mean|  3.9266949863509617|
|    Dataset|              *|        Size|           3522285.0|
|     Column|   payment_type|  Compliance|  0.9560146325467701|
|     Column|       VendorID|Completeness|                 1.0|
|Mutlicolumn|VendorID,gen_id|  Uniqueness|                 1.0|
|     Column|          extra|     Maximum|               17.25|
+-----------+---------------+------------+--------------------+

In [39]:
%%display
analysisResult_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [38]:
checkResult = VerificationSuite(spark) \
    .onData(sample_df) \
    .addCheck(
        check.isComplete("VendorID")  \
        .isUnique("gen_id")  \
        .hasUniqueness(["VendorID", "gen_id"], assertion=lambda x: x==1) \
        .hasCompleteness("Airport_fee", assertion=lambda x: x >= 0.95) \
        .isNonNegative("fare_amount")) \
        .useRepository(nyc_yellow_repository) \
        .saveOrAppendResult(verify_resultKey) \
    .run()

print(f"Verification Run Status: {checkResult.status}")
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Verification Run Status: Warning
+--------------------+-----------+------------+--------------------+-----------------+--------------------+
|               check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+--------------------+-----------+------------+--------------------+-----------------+--------------------+
|NYC Yellow Taxi T...|    Warning|     Warning|CompletenessConst...|          Success|                    |
|NYC Yellow Taxi T...|    Warning|     Warning|UniquenessConstra...|          Success|                    |
|NYC Yellow Taxi T...|    Warning|     Warning|UniquenessConstra...|          Success|                    |
|NYC Yellow Taxi T...|    Warning|     Warning|CompletenessConst...|          Success|                    |
|NYC Yellow Taxi T...|    Warning|     Warning|ComplianceConstra...|          Failure|Value: 0.98946706...|
|NYC Yellow Taxi T...|    Warning|     Warning|CompletenessConst...|          Success|                 

In [40]:
%%display
checkResult_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

### Reading metrics from Metrics Repository

In [42]:
ny_yellow_analysis_result = nyc_yellow_repository.load() \
                                .before(ResultKey.current_milli_time()) \
                                .getSuccessMetricsAsDataFrame()

ny_yellow_analysis_result.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+------------+--------------------+-------------+--------------------+
|     entity|            instance|        name|               value| dataset_date|                 tag|
+-----------+--------------------+------------+--------------------+-------------+--------------------+
|     Column|        total_amount|         Sum|1.0274988166003421E8|1703334750003| nyc_yellow_oct_2023|
|     Column|            VendorID|Distinctness|8.517198352773839E-7|1703334750003| nyc_yellow_oct_2023|
|     Column|       trip_distance|        Mean|  3.9266949863509617|1703334750003| nyc_yellow_oct_2023|
|    Dataset|                   *|        Size|           3522285.0|1703334750003| nyc_yellow_oct_2023|
|     Column|        payment_type|  Compliance|  0.9560146325467701|1703334750003| nyc_yellow_oct_2023|
|     Column|            VendorID|Completeness|                 1.0|1703334750003| nyc_yellow_oct_2023|
|Mutlicolumn|     VendorID,gen_id|  Uniqueness|                 

In [43]:
%%display
ny_yellow_analysis_result

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [44]:
ny_yellow_verification = nyc_yellow_repository.load().withTagValues(verify_key_tag).getSuccessMetricsAsDataFrame()
ny_yellow_verification.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+------------+------------------+-------------+--------------------+
|     entity|            instance|        name|             value| dataset_date|                 tag|
+-----------+--------------------+------------+------------------+-------------+--------------------+
|     Column|fare_amount is no...|  Compliance|0.9894670647037364|1703334750010|verify_nyc_yellow...|
|     Column|              gen_id|  Uniqueness|               1.0|1703334750010|verify_nyc_yellow...|
|     Column|         Airport_fee|Completeness|0.9560146325467701|1703334750010|verify_nyc_yellow...|
|     Column|            VendorID|Completeness|               1.0|1703334750010|verify_nyc_yellow...|
|Mutlicolumn|     VendorID,gen_id|  Uniqueness|               1.0|1703334750010|verify_nyc_yellow...|
+-----------+--------------------+------------+------------------+-------------+--------------------+

In [45]:
%%display
ny_yellow_verification

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [46]:
%%display
checkResult_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

## Writing Verification Results into Athena Table

In [48]:
checkResult_df.write.insertInto("nyc_tlc.pydeequ_verification_results")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Reading metrics file from S3

In [ ]:
# metrics JSON file S3 Path
metrics_s3_path = "s3://{bucket_name}/nyc_tlc/pydeequ_metrics/ny_yellow_metrics.json"

repository_from_s3 = FileSystemMetricsRepository(spark, path = metrics_s3_path)

metrics_df = repository_from_s3.load() \
                                .before(ResultKey.current_milli_time()) \
                                .getSuccessMetricsAsDataFrame()
metrics_df.show()